### preqs
```bash
# saai
$ start nlu_multilang

# agent-dispatcher
$ start servant_nlu
$ start agent
$ start actions
```

In [1]:
import requests
from pprint import pprint
def talk(sents, sender='default', port=5005):
    response = requests.post(f'http://localhost:{port}/webhooks/rest/webhook',
                             json={'message': sents, 'sender':sender})
    print('status code:', response.status_code)
    pprint(response.json())
    
talk('hi')

status code: 200
[{'recipient_id': 'default', 'text': 'Hey! How are you?'}]


* 这是结合rasa-agent, ruleset, rasa-nlu的调用案例
* 'list some products'会被推断到意图list_products, 而这个意图会触发action: action_list_products, 而这个action会返回包含product_list字段的数据
* 实际的应用会连接到backend(比如odoo或ofbiz), 来获取数据库中的真实数据.

In [3]:
talk('list some products')

status code: 200
[{'custom': {'product_list': ['first prod', 'second prod'],
             'result': 'success'},
  'recipient_id': 'default'}]


In [3]:
# preq: 
# $ rasa run --enable-api -v
# $ rasa run actions

port=5005
values={"object_type": 'music',
        "sents":'hi',
        'parameters': []}
values_str=json.dumps(values, ensure_ascii=False)
# text = f'/{ac["intent"]}{{"object_type": "{ac["object_type"]}", "sents":"{ac["sents"]}"}}'
text = f'/ask_perform_media{values_str}'
data = {'sender': 'default', "message": text}
response = requests.post(f'http://localhost:{port}/webhooks/rest/webhook', json=data)
# print('status code:', response.status_code)
if response.status_code==200:
    pprint(response.json())

[{'custom': {'media_list': ['first song', 'second song'],
             'media_type': 'music',
             'result': 'success',
             'sents': 'hi'},
  'recipient_id': 'default'}]


* 使用http-api来直接调用对应act_products意图的action, 这种方式对于测试action比较方便
* 如果需要携带参数, 可以在message之后附带上就可以了, 参数值在action里可以用get_slot得到
* 因为在train的时候去掉了keras策略, 所以train的过程非常快(因为没有lstm神经网络的训练时耗), 等到actions测试完成后, 可以再加上keras策略

In [6]:
import requests
from pprint import pprint

port=5005
text = f'/act_products'
data = {'sender': 'default', "message": text}
response = requests.post(f'http://localhost:{port}/webhooks/rest/webhook', json=data)
# print('status code:', response.status_code)
if response.status_code==200:
    pprint(response.json())

[{'custom': {'product_list': [{'id': 38, 'name': '[CONS_25630] Screw'},
                              {'id': 37, 'name': '[CONS_89957] Bolt'},
                              {'id': 13,
                               'name': '[DESK0004] Customizable Desk '
                                       '(Aluminium, Black)'},
                              {'id': 49,
                               'name': '[DESK0005] Customizable Desk (Custom, '
                                       'White)'},
                              {'id': 50,
                               'name': '[DESK0006] Customizable Desk (Custom, '
                                       'Black)'},
                              {'id': 47, 'name': '[Deposit] Deposit'},
                              {'id': 14,
                               'name': '[E-COM06] Corner Desk Right Sit'},
                              {'id': 15, 'name': '[E-COM07] Large Cabinet'},
                              {'id': 16, 'name': '[E-COM08] Storage Box'},
  